# AAA - kurzushétvége - Lineáris modell rész

In [ ]:
#adatok betöltése

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import time
import glob
from pathlib import Path
from tqdm import tqdm_notebook as tqdm

In [2]:
data_folder = Path("../2_data_manipulation/data/*.csv")

In [3]:
files = glob.glob(str(data_folder))

In [4]:
li = []


for filename in tqdm(files):
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)


autok_osszes_df = pd.concat(li, axis=0, ignore_index=True)

/usr/lib/python3/dist-packages/ipykernel_launcher.py:9: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  if __name__ == '__main__':


In [ ]:
#A statikus df kialakítása

autok_statikus_df = autok_osszes_df.drop(['price','price-parsed','datum'], axis=1).drop_duplicates()

###Kész!

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.


KeyboardInterrupt

ERROR:tornado.general:Uncaught exception, closing connection.
Traceback (most recent call last):
  File "/usr/lib/python3/dist-packages/zmq/eventloop/zmqstream.py", line 414, in _run_callback
    callback(*args, **kwargs)
  File "/usr/lib/python3/dist-packages/tornado/stack_context.py", line 277, in null_wrapper
    return fn(*args, **kwargs)
  File "/usr/lib/python3/dist-packages/ipykernel/kernelbase.py", line 283, in dispatcher
    return self.dispatch_shell(stream, msg)
  File "/usr/lib/python3/dist-packages/ipykernel/kernelbase.py", line 233, in dispatch_shell
    handler(stream, idents, msg)
  File "/usr/lib/python3/dist-packages/ipykernel/kernelbase.py", line 421, in execute_request
    self._abort_queues()
  File "/usr/lib/python3/dist-packages/ipykernel/kernelbase.py", line 636, in _abort_queues
    self._abort_queue(stream)
  File "/usr/lib/python3/dist

In [5]:
def idokinyero(df):
    #df['datum'] = pd.to_datetime(df['datum'])
    df_s = df.sort_values(by = ['datum'])
    return pd.Series({'eslo_nap': df_s['datum'].min(), 'utolso_nap': df_s['datum'].max(),
                      'eltelt_napok': (df_s['datum'].max()-df_s['datum'].min()).days,
                      'kezdeti_ar': df_s['price-parsed'].iloc[0], 'vegso_ar': df_s['price-parsed'].iloc[-1]})

In [ ]:
auto_idoben_valtozo_df = autok_osszes_df[['link','datum','price-parsed']]

In [ ]:
auto_idoben_valtozo_df['datum'] = pd.to_datetime(auto_idoben_valtozo_df['datum'])

In [ ]:
%%time
auto_idoinfo_df = auto_idoben_valtozo_df.groupby('link').apply(idokinyero)

In [ ]:
auto_idoinfo_df['arkulonbseg'] = auto_idoinfo_df['vegso_ar'] - auto_idoinfo_df['kezdeti_ar']

In [ ]:
#auto_idoinfo_df['arkulonbseg'].value_counts()
#auto_idoinfo_df.head(10)
#gb_dic['eltelt_napok'].iloc[-1]

In [ ]:
auto_idoinfo_df['sold'] = np.where(auto_idoinfo_df['utolso_nap'] < auto_idoinfo_df['utolso_nap'].max(), 1,0 )

auto_idoinfo_df['eladasig_nap'] = np.where(auto_idoinfo_df['sold'] == 1,auto_idoinfo_df['eltelt_napok'], np.nan )

auto_idoinfo_df['piacon_nap'] = np.where(auto_idoinfo_df['sold'] == 0, auto_idoinfo_df['eltelt_napok'], np.nan )



In [ ]:
auto_adat_kesz = autok_statikus_df.merge(auto_idoinfo_df.reset_index(), on='link')

In [ ]:
auto_adat_kesz.head(10)

In [ ]:
output_folder = Path("data/")

In [ ]:
auto_adat_kesz.to_csv(str(output_folder) + 'auto_data_model.csv')